In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Defining a Neural Network in PyTorch
====================================

Deep learning uses artificial neural networks (models), which are
computing systems that are composed of many layers of interconnected
units. By passing data through these interconnected units, a neural
network is able to learn how to approximate the computations required to
transform inputs into outputs. In PyTorch, neural networks can be
constructed using the `torch.nn` package.

Introduction
------------

PyTorch provides the elegantly designed modules and classes, including
`torch.nn`, to help you create and train neural networks. An `nn.Module`
contains layers, and a method `forward(input)` that returns the
`output`.

In this recipe, we will use `torch.nn` to define a neural network
intended for the [MNIST
dataset](hhttps://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST).

Setup
-----

Before we begin, we need to install `torch` if it isn't already
available.

    pip install torch


Steps
=====

1.  Import all necessary libraries for loading our data
2.  Define and initialize the neural network
3.  Specify how data will pass through your model
4.  \[Optional\] Pass data through your model to test

1. Import necessary libraries for loading our data
--------------------------------------------------

For this recipe, we will use `torch` and its subsidiaries `torch.nn` and
`torch.nn.functional`.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F #F 是 PyTorch 的 函数式接口（来自 torch.nn.functional 模块）

2. Define and initialize the neural network
===========================================

Our network will recognize images. We will use a process built into
PyTorch called convolution. Convolution adds each element of an image to
its local neighbors, weighted by a kernel, or a small matrix, that helps
us extract certain features (like edge detection, sharpness, blurriness,
etc.) from the input image.

There are two requirements for defining the `Net` class of your model.
The first is writing an \_\_init\_\_ function that references
`nn.Module`. This function is where you define the fully connected
layers in your neural network.

Using convolution, we will define our model to take 1 input image
channel, and output match our target of 10 labels representing numbers 0
through 9. This algorithm is yours to create, we will follow a standard
MNIST algorithm.


In [7]:
class Net(nn.Module):
    def __init__(self):
      '''
      The first is writing an __init__ function that references nn.Module.
      This function is where you define the fully connected layers in your neural network.
      '''

      super(Net, self).__init__() #必须的一句

      # First 2D convolutional layer, taking in 1 input channel (image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      #MNIST数据 28x28
      #self.conv1 = nn.Conv2d(1, 32, 3, 1)
      #1：通道(灰度图)、32:32通道(提取32种特征)、3:(卷积核3x3)、1:步长、同时无填充padding=0
      #第一次卷积层后 输出size减少2像素 28x28 > 26x26

      # Second 2D convolutional layer, taking in the 32 input layers,
      # outputting 64 convolutional features, with a square kernel size of 3
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      # 26x26 >24x24 通道由32 > 64

      # Designed to ensure that adjacent pixels are either all 0s or all active
      # with an input probability
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # First fully connected layer
      # 全连接层: input:24x24x64x0.25
      self.fc1 = nn.Linear(9216, 128)
      # Second fully connected layer that outputs our 10 labels
      self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


We have finished defining our neural network, now we have to define how
our data will pass through it.

3. Specify how data will pass through your model
================================================

When you use PyTorch to build a model, you just have to define the
`forward` function, that will pass the data into the computation graph
(i.e. our neural network). This will represent our feed-forward
algorithm.

You can use any of the Tensor operations in the `forward` function.


In [8]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x) #输入：1x28x28 → 输出：32x26x26（计算公式：28−3+1=26）
      # Use the rectified-linear activation function over x
      x = F.relu(x)
      #F方法:
      #提供直接调用的函数（如激活函数、池化、损失计算等），不保存可训练参数
      #允许在模型前向传播中动态控制操作（如手动定义计算流程）。

      x = self.conv2(x) #输入：32x26x26 → 输出：64x24x24（计算公式：26−3+1=24）
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      #最大池化 输入：64x24x24 → 输出：64x12x12（池化核 2x2，步长 2）

      '''
      max_pool1d用于一维数据，如时间序列或文本，处理的是长度维度。例如，输入形状为(batch_size, channels, length)，池化在length维度上进行。
      max_pool2d用于二维数据，如图像，处理高度和宽度。输入形状为(batch_size, channels, height, width)，池化核是二维的，常见于卷积神经网络中的空间下采样。
      max_pool3d处理三维数据，如视频或体积数据，输入形状为(batch_size, channels, depth, height, width)，池化在三个空间维度上进行。

      '''

      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through ``fc1``
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      #卷积 → 激活 → 卷积 → 激活 → 池化 → Dropout → 展平 → 全连接 → 激活 → Dropout → 输出
      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      #使用 log_softmax 配合负对数似然损失（NLLLoss），适用于分类任务。
      # 计算对数概率（dim=1 表示对类别维度操作）
      return output

4. \[Optional\] Pass data through your model to test
====================================================

To ensure we receive our desired output, let's test our model by passing
some random data through it.


In [10]:
# Equates to one random 28x28 image
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)# 做的是F.log_softmax(x, dim=1)  对数softmax 输出的结果是每个类别对数后结果，相加是不为1的

tensor([[-2.4235, -2.2761, -2.2490, -2.2621, -2.3828, -2.3694, -2.2343, -2.3663,
         -2.1808, -2.3086]], grad_fn=<LogSoftmaxBackward0>)


Each number in this resulting tensor equates to the prediction of the
label the random tensor is associated to.

Congratulations! You have successfully defined a neural network in
PyTorch.

Learn More
==========

Take a look at these other recipes to continue your learning:

-   [What is a state\_dict in
    PyTorch](https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html)
-   [Saving and loading models for inference in
    PyTorch](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html)
